# Breadth Indicators

Breadth Indicators include:

Chaikin Oscillator

Up/Down Volume Ratio

Up/Down Volume Spread

Cumulative Volume Index

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

# fix_yahoo_finance is used to fetch data 
import yfinance as yf
yf.pdr_override()

In [3]:
# input
symbol = '^GSPC'
start = '2020-01-01'
end = '2023-01-01'

# Read data 
df = yf.download(symbol,start,end)

# View Columns
df.head()

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2020-01-02 00:00:00-05:00,3244.669922,3258.139893,3235.530029,3257.850098,3257.850098,3459930000
2020-01-03 00:00:00-05:00,3226.360107,3246.149902,3222.340088,3234.850098,3234.850098,3484700000
2020-01-06 00:00:00-05:00,3217.550049,3246.840088,3214.639893,3246.280029,3246.280029,3702460000
2020-01-07 00:00:00-05:00,3241.860107,3244.909912,3232.429932,3237.179932,3237.179932,3435910000
2020-01-08 00:00:00-05:00,3238.590088,3267.070068,3236.669922,3253.050049,3253.050049,3726840000


In [4]:
df['Adj Close'][1:]

Date
2020-01-03 00:00:00-05:00    3234.850098
2020-01-06 00:00:00-05:00    3246.280029
2020-01-07 00:00:00-05:00    3237.179932
2020-01-08 00:00:00-05:00    3253.050049
2020-01-09 00:00:00-05:00    3274.699951
                                ...     
2022-12-23 00:00:00-05:00    3844.820068
2022-12-27 00:00:00-05:00    3829.250000
2022-12-28 00:00:00-05:00    3783.219971
2022-12-29 00:00:00-05:00    3849.280029
2022-12-30 00:00:00-05:00    3839.500000
Name: Adj Close, Length: 755, dtype: float64

In [9]:
import talib as ta

https://www.investopedia.com/terms/m/mcclellanoscillator.asp

In [11]:
import quandl as q

# For NASDAQ
#Advances = q.get('URC/NASDAQ_ADV')['Numbers of Stocks']
#Declines = q.get('URC/nASDAQ_DEC')['Numbers of Stocks']   
#n = Advances - Declines

In [12]:
Advances = q.get('URC/NYSE_ADV', start_date = "2020-01-01")['Numbers of Stocks']
Declines = q.get('URC/NYSE_DEC', start_date = "2023-01-01")['Numbers of Stocks']  

In [13]:
adv_vol = q.get("URC/NYSE_ADV_VOL", start_date = "2020-01-01")['Numbers of Stocks']
dec_vol = q.get("URC/NYSE_DEC_VOL", start_date = "2023-01-01")['Numbers of Stocks']

In [14]:
data = pd.DataFrame()
data['Advances'] = Advances
data['Declines'] = Declines
data['adv_vol'] = adv_vol
data['dec_vol'] = dec_vol

In [27]:
data['Net_Advances'] = data['Advances'] - data['Declines'] 
data['Ratio_Adjusted'] = (data['Net_Advances']/(data['Advances'] + data['Declines'])) * 1000
data['19_EMA'] = ta.EMA(data['Ratio_Adjusted'], timeperiod=19)
data['39_EMA'] = ta.EMA(data['Ratio_Adjusted'], timeperiod=39)
data['RANA'] = (data['Advances'] - data['Declines']) / (data['Advances'] + data['Declines']) * 1000

Exception: inputs are all NaN

In [ ]:
# Finding the TRIN Value
data['ad_ratio'] = data['Advances'].divide(data['Declines'] ) # AD Ratio
data['ad_vol'] = data['adv_vol'].divide(data['dec_vol']) # AD Volume Ratio
data['TRIN'] = data['ad_ratio'].divide(data['adv_vol']) # TRIN Value

In [ ]:
data.head()

## Chaikin Oscillator

In [24]:
def Chaikin(data):
    money_flow_volume = (2 * df['Adj Close'] - df['High'] - df['Low']) / (df['High'] - df['Low']) * df['Volume']  
    ad = money_flow_volume.cumsum()
    Chaikin = pd.Series(ad.ewm(com=(3-1)/2).mean() - ad.ewm(com=(10-1)/2).mean(), name='Chaikin')
    data = data.join(Chaikin)  
    return data

In [25]:
Chaikin(df)

,Open,High,Low,Close,Adj Close,Volume,Chaikin
Date,,,,,,,
2020-01-02 00:00:00-05:00,3244.669922,3258.139893,3235.530029,3257.850098,3257.850098,3459930000,0.000000e+00
2020-01-03 00:00:00-05:00,3226.360107,3246.149902,3222.340088,3234.850098,3234.850098,3484700000,2.066404e+07
2020-01-06 00:00:00-05:00,3217.550049,3246.840088,3214.639893,3246.280029,3246.280029,3702460000,6.278656e+08
2020-01-07 00:00:00-05:00,3241.860107,3244.909912,3232.429932,3237.179932,3237.179932,3435910000,5.711566e+08
2020-01-08 00:00:00-05:00,3238.590088,3267.070068,3236.669922,3253.050049,3253.050049,3726840000,5.521871e+08
...,...,...,...,...,...,...,...
2022-12-23 00:00:00-05:00,3815.110107,3845.800049,3797.010010,3844.820068,3844.820068,2819280000,5.445932e+08
2022-12-27 00:00:00-05:00,3843.340088,3846.649902,3813.219971,3829.250000,3829.250000,3030300000,1.000376e+09
2022-12-28 00:00:00-05:00,3829.560059,3848.320068,3780.780029,3783.219971,3783.219971,3083520000,1.856573e+08


## Up/Down Volume Ratio

Volume Spread = Up Volume - Down Volume 

In [22]:
Up = q.get('URC/NYSE_ADV', start_date = "2017-07-27")['Numbers of Stocks']
Down = q.get('URC/NYSE_DEC', start_date = "2017-07-27")['Numbers of Stocks']
Volume_Spread = Up - Down

In [23]:
Volume_Spread

Date
2017-07-27   -290.0
2017-07-28    238.0
2017-07-31    183.0
2017-08-01    605.0
2017-08-02   -592.0
              ...  
2020-02-10    509.0
2020-02-11      0.0
2020-02-12      0.0
2020-02-13      0.0
2020-02-14      0.0
Name: Numbers of Stocks, Length: 647, dtype: float64

## Up/Down Volume Spread

Volume Ratio = Up Volume / Down Volume

In [20]:
Up = q.get('URC/NYSE_ADV', start_date = "2017-07-27")['Numbers of Stocks']
Down = q.get('URC/NYSE_DEC', start_date = "2017-07-27")['Numbers of Stocks']
Volume_Ratio = Up/Down

In [21]:
Volume_Ratio

Date
2017-07-27    0.819876
2017-07-28    1.176689
2017-07-31    1.130714
2017-08-01    1.517536
2017-08-02    0.664399
                ...   
2020-02-10    1.419967
2020-02-11         NaN
2020-02-12         NaN
2020-02-13         NaN
2020-02-14         NaN
Name: Numbers of Stocks, Length: 647, dtype: float64

## Cumulative Volume Index

https://www.marketinout.com/technical_analysis.php?t=Cumulative_Volume_Index_(CVI)&id=38

In [18]:
# CVI = Yesterday's CVI + (Advancing Volume - Declining Volume)
data['CVI'] = data['Net_Advances'][1:] + (data['Advances'] - data['Declines']) 

In [19]:
data.head()

,Advances,Declines,adv_vol,dec_vol,Net_Advances,Ratio_Adjusted,CVI
Date,,,,,,,
2020-01-02,1808.0,NaN,1.980000e+09,NaN,NaN,NaN,NaN
2020-01-03,1387.0,NaN,1.500000e+09,NaN,NaN,NaN,NaN
2020-01-06,1557.0,NaN,2.130000e+09,NaN,NaN,NaN,NaN
2020-01-07,1290.0,NaN,1.610000e+09,NaN,NaN,NaN,NaN
2020-01-08,1744.0,NaN,1.940000e+09,NaN,NaN,NaN,NaN
